In [19]:
from dataclasses import dataclass
import pandas as pd
%matplotlib inline
import random
import statsmodels.api as sm

In [20]:
@dataclass
class ModelInputs:
    başlangıç_maaşı: int = 502000
    terfi_sıklığı: int = 5
    yaşam_maliyeti_artışı: float = 0.02
    terfi_zam_oranı: float = 0.15
    biriktirme_oranı: float = 0.25
    faiz_oranı: float = 0.05
    hedef_nakit: int = 1500000
        
model_verileri = ModelInputs()

In [32]:
@dataclass
class SimulasyonVerileri:
    n_iterasyonlar: int = 10000
    başlangıç_maaşı_std: int = 10000
    terfi_sıklığı_std: int = 1.5
    yaşam_maliyeti_artışı_std: float = 0.005
    terfi_zam_oranı_std: float = 0.05
    biriktirme_oranı_std: float = 0.07
    faiz_oranı_std: float = 0.01
        
sim_verileri = SimulasyonVerileri()



def yıldaki_maaş(data: ModelInputs, yıl):
    """
    Başlangıçtan itibaren yaşam maliyeti artışlarına ve düzenli terfilerine dayalı olarak belirli bir yıldaki maaşı elde eder.
    """
    # Her n yılda bir terfi yapılır, bu nedenle yılları böler ve ondalıkları çıkartarak terfi sayısını elde ederiz
    terfi_sayısı = int(yıl / data.terfi_sıklığı)
    
    # Yukarıdaki formülü Python'da uyguladık
    maaş_t = data.başlangıç_maaşı * (1 + data.yaşam_maliyeti_artışı) ** yıl * (1 + data.terfi_zam_oranı) ** terfi_sayısı
    return maaş_t

def yıl_boyunca_biriktirilen_nakit(data: ModelInputs, yıl):
    """
    Belirli bir yılda biriktirilen nakit miktarını hesaplar. İlk olarak, o yıldaki maaşı hesaplayarak birikim oranını
    uygular.
    """
    maaş = yıldaki_maaş(data, yıl)
    biriktirilen_nakit = maaş * data.biriktirme_oranı
    return biriktirilen_nakit


def yıldaki_varlık(data: ModelInputs, yıl, önceki_varlık):
    """
    Önceki yıldaki varlığa, yatırım oranına ve o yıl boyunca biriktirilen nakit miktarına dayanarak belirli bir yıl için varlığı hesaplar.
    """
    biriktirilen_nakit = yıl_boyunca_biriktirilen_nakit(data, yıl)
    varlık = önceki_varlık * (1 + data.faiz_oranı) + biriktirilen_nakit
    return varlık

def emeklilik_yılları(data: ModelInputs, çıktıyı_yazdır=True):
    
    # başlangıçta biriktirilmiş nakit yok
    önceki_varlık = 0  
    varlık = 0
    
    yıl = 0  # ilk döngüde 1 olacak
    
    if çıktıyı_yazdır:
        print('Varlıklar zaman içinde:') # \n çıktıda boş bir satır oluşturur.
    while varlık < data.hedef_nakit:
        yıl = yıl + 1
        varlık = yıldaki_varlık(data, yıl, önceki_varlık)
        if çıktıyı_yazdır:
            print(f'{yıl}. yıldaki varlık {varlık:,.0f} TL.')

        # Gelecek yılın önceki varlığı bu yılın varlığı olarak ayarlanır
        önceki_varlık = varlık
        
    # Şimdi while döngüsünden çıktık, bu yüzden varlık >= hedef_nakit olmalı. Son belirlenen yıl,
    # emeklilik için geçecek yılları temsil eder.
    if çıktıyı_yazdır:
        print(f'\nEmeklilik:\nEmekli olmak için {yıl} yıl gerekecek.')  #  \n çıktıda boş bir satır oluşturur.
    return yıl


def rastgele_normal_pozitif(ortalama, std):
    """
    Bu fonksiyon, pozitif bir sayı elde edene kadar normal dağılımdan rastgele sayılar seçmeye devam eder,
    ardından o sayıyı döndürür.
    """
    çekilen_değer = -1  # while döngüsünün başlaması için başlangıçta negatif bir sayıya başlayın
    while çekilen_değer < 0:
        çekilen_değer = random.normalvariate(ortalama, std)
    return çekilen_değer


def emeklilik_simulasyonu_girdileri(veri, sim_veri):
    """
    Aşağıdaki girdileri, ModelInputs değerlerini ortalama olarak kullanarak ve SimulasyonVerileri'ndeki
    _std değerlerini standart sapma olarak kullanarak normal dağılımlardan rastgele seçer:
    - Başlangıç maaşı
    - Terfi sıklığı
    - Yaşam maliyeti artışı
    - Terfi zam oranı
    - Biriktirme oranı
    - Faiz oranı
    

    Ayrıca, herhangi bir seçilen değer sıfır veya daha küçükse, yeniden seçilir.
    """
    başlangıç_maaşı = rastgele_normal_pozitif(veri.başlangıç_maaşı, sim_veri.başlangıç_maaşı_std)
    terfi_sıklığı = rastgele_normal_pozitif(veri.terfi_sıklığı, sim_veri.terfi_sıklığı_std)
    yaşam_maliyeti_artışı = rastgele_normal_pozitif(veri.yaşam_maliyeti_artışı, sim_veri.yaşam_maliyeti_artışı_std)
    terfi_zam_oranı = rastgele_normal_pozitif(veri.terfi_zam_oranı, sim_veri.terfi_zam_oranı_std)
    biriktirme_oranı = rastgele_normal_pozitif(veri.biriktirme_oranı, sim_veri.biriktirme_oranı_std)
    faiz_oranı = rastgele_normal_pozitif(veri.faiz_oranı, sim_veri.faiz_oranı_std)
  
    

    return (
        başlangıç_maaşı,
        terfi_sıklığı,
        yaşam_maliyeti_artışı,
        terfi_zam_oranı,
        biriktirme_oranı,
        faiz_oranı,
      
    )

def tekli_emeklilik_simulasyonu(veri, sim_veri):
    """
    Emeklilik için yılın tek bir Monte Carlo simülasyonunu çalıştırır.
    
    emeklilik_simulasyonu_girdileri fonksiyonunu kullanır.
    """
    # Girdi değerlerini normal dağılımlardan seç
    (
        başlangıç_maaşı,
        terfi_sıklığı,
        yaşam_maliyeti_artışı,
        terfi_zam_oranı,
        biriktirme_oranı,
        faiz_oranı,
        
    ) = emeklilik_simulasyonu_girdileri(veri, sim_veri)
    
    # Model girdilerini oluştur
    yeni_veri = ModelInputs(
        başlangıç_maaşı=başlangıç_maaşı,
        terfi_sıklığı=terfi_sıklığı,
        yaşam_maliyeti_artışı=yaşam_maliyeti_artışı,
        terfi_zam_oranı=terfi_zam_oranı,
        biriktirme_oranı=biriktirme_oranı,
        faiz_oranı=faiz_oranı,
        
    )
    
    # Modeli çalıştır
    emeklilik_yılı = emeklilik_yılları(yeni_veri, çıktıyı_yazdır=False)
    
    return (
        başlangıç_maaşı,
        terfi_sıklığı,
        yaşam_maliyeti_artışı,
        terfi_zam_oranı,
        biriktirme_oranı,
        faiz_oranı,
        emeklilik_yılı
  
    )

tekli_emeklilik_simulasyonu(model_verileri, sim_verileri)

(87064.07154572179,
 5.743188269268792,
 0.018363722645176554,
 0.11930897115677717,
 0.27502370527288916,
 0.04585639739044066,
 25)

In [61]:
def emeklilik_yılı_mc(veri, sim_veri):
    """
    SimülasyonVerileri'ndeki n_iterasyonlar için tekil_emeklilik_simulasyonu kullanarak tam Monte Carlo 
    simülasyonunu çalıştırır.

    Girdi değerlerini ve emeklilik için geçecek yıl sayısını içeren bir DataFrame çıktısı üretir.
    """
    tüm_sonuçlar = [tekli_emeklilik_simulasyonu(veri, sim_veri) for _ in range(sim_veri.n_iterasyonlar)]
    df = pd.DataFrame(
        tüm_sonuçlar,
        columns=[
            'Başlangıç Maaşı',
            'Terfi Sıklığı',
            'Yaşam Maliyeti Artışı',
            'Terfi Zam Oranı',
            'Biriktirme Oranı',
            'Faiz Oranı',
            'Emeklilik için Geçecek Yıllar'
        ]
    )
    return df

df = emeklilik_yılı_mc(model_verileri, sim_verileri)
def stilize_edilmis_df(df):
    """
    Girdileri ve emeklilik için geçecek yılları içeren DataFrame'leri stilize eder..
    """
    return df.style.format({
        'Başlangıç Maaşı': '{:,.0f} TL', 
        'Terfi Sıklığı': '{:.1f} Yıl', 
        'Yaşam Maliyeti Artışı': '{:.2%}', 
        'Terfi Zam Oranı': '{:.2%}', 
        'Biriktirme Oranı': '{:.2%}', 
        'Faiz Oranı': '{:.2%}',
        'Emeklilik için Geçecek Yıllar': '{:.0f}'
    }).background_gradient(cmap='RdYlGn_r', subset='Emeklilik için Geçecek Yıllar')

In [62]:
stilize_edilmis_df(df.head())

,Başlangıç Maaşı,Terfi Sıklığı,Yaşam Maliyeti Artışı,Terfi Zam Oranı,Biriktirme Oranı,Faiz Oranı,Emeklilik için Geçecek Yıllar
0,"506,099 TL",0.7 Yıl,2.02%,12.89%,19.86%,4.66%,7
1,"513,177 TL",2.4 Yıl,2.04%,11.17%,11.43%,6.59%,13
2,"497,072 TL",1.3 Yıl,1.90%,12.54%,17.00%,4.82%,10
3,"497,548 TL",1.9 Yıl,2.71%,14.96%,21.31%,4.81%,8
4,"484,629 TL",1.2 Yıl,2.06%,19.94%,19.76%,5.40%,8


In [59]:
@dataclass
class ModelInputs:
    başlangıç_maaşı: int = 500000
    terfi_sıklığı: int = 2
    yaşam_maliyeti_artışı: float = 0.02
    terfi_zam_oranı: float = 0.15
    biriktirme_oranı: float = 0.25
    faiz_oranı: float = 0.05
    hedef_nakit: int = 1500000
        
model_verileri = ModelInputs()

In [63]:
quants = df.quantile([i / 20 for i in range(1, 20)])
stilize_edilmis_df(quants)

,Başlangıç Maaşı,Terfi Sıklığı,Yaşam Maliyeti Artışı,Terfi Zam Oranı,Biriktirme Oranı,Faiz Oranı,Emeklilik için Geçecek Yıllar
0.050000,"483,513 TL",0.3 Yıl,1.18%,6.78%,13.47%,3.38%,4
0.100000,"487,211 TL",0.6 Yıl,1.35%,8.57%,16.08%,3.74%,5
0.150000,"489,558 TL",0.9 Yıl,1.48%,9.71%,17.68%,3.98%,6
0.200000,"491,538 TL",1.1 Yıl,1.58%,10.66%,19.06%,4.17%,6
0.250000,"493,201 TL",1.3 Yıl,1.66%,11.47%,20.27%,4.33%,7
0.300000,"494,635 TL",1.5 Yıl,1.74%,12.26%,21.30%,4.48%,7
0.350000,"495,963 TL",1.7 Yıl,1.81%,12.92%,22.26%,4.62%,7
0.400000,"497,351 TL",1.8 Yıl,1.87%,13.59%,23.14%,4.75%,7
0.450000,"498,556 TL",2.0 Yıl,1.93%,14.25%,24.04%,4.87%,8
0.500000,"499,821 TL",2.2 Yıl,1.99%,14.94%,24.95%,5.00%,8
